# ML Insights Tests

## Use Case
This notebook shows how to configure and write Tests using ML Insights Library.
The Tests can be used apply threshold on the metrics. This will allow to continously monitor the metrics of the data.

## Notes

Insights Support following type of tests:
1. Predicated Based test: The test take two value and a condition and evaluates the expression.
2. Metric Based test: These test are specific to a particular metrics.


Supported Predicate based tests:
1. TestGreaterThan
2. TestLessThan
3. TestEqual
4. TestDeviation
5. TestIsBetween
6. TestStringEquals


Supports Metric based tests:
1. TestIsPositive
2. TestIsNegative
3. TestIsNonZero
4. TestIsComplete
5. TestIsMatchingInferenceType
6. TestNoNewCategory


## About Dataset

The data was collected and made available by “National Institute of Diabetes and Digestive and Kidney Diseases” as part of the Pima Indians Diabetes Database. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here belong to the Pima Indian heritage (subgroup of Native Americans), and are females of ages 21 and above.

The data set contains medical and demographic data of patients . It consists of various features such as Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome, Prediction, BMICategory, Prediction_Score .

Dataset source : https://www.kaggle.com/datasets/kandij/diabetes-dataset

# Install ML Observability Insights Library SDK

- Prerequisites
    - Linux/Mac (Intel CPU)
    - Python 3.8 and 3.9 only


- Installation
    - ML Insights is made available as a Python package (via Artifactory) which can be installed using pip install as shown below. Depending on the execution engine on which to do the run, one can use scoped package. For eg: if we want to run on dask, use oracle-ml-insights[dask], for spark use oracle-ml-insights[spark], for native use oracle-ml-insights. One can install all the dependencies as use oracle-ml-insights[all]

      !pip install oracle-ml-insights


Refer : [Installation and Setup](https://docs.oracle.com/en-us/iaas/tools/ml-insights-docs/latest/ml-insights-documentation/html/user_guide/tutorials/install.html)

In [1]:
# !python3 -m pip install oracle-ml-insights

## Generating a Reference Profile[Optional]

Use of refernce profile writing tests is optional, however to compare the current metrics to with a metrics from previous run, it is necessary to load the previos profile

In [2]:
from mlm_insights.config_reader.insights_config_reader import InsightsConfigReader
monitor_config_path = "monitor_configs/monitor_config_diabetes_reference.json"
builder = InsightsConfigReader(config_location=monitor_config_path).get_builder().build()
reference_profile = builder.run().profile

reference_profile_location = "output_data/profiles/profile_diabetes_reference.bin"
with open(reference_profile_location, "wb") as file:
    file.write(reference_profile.marshall())

## Generating/Loading a Current Profile

If a profile already exist, then load the profile on which test will be performed,
or create a  new profile.

In [3]:
monitor_config_path = "monitor_configs/monitor_config_diabetes_current.json"
builder = InsightsConfigReader(config_location=monitor_config_path).get_builder().build()
current_profile = builder.run().profile

# Creating a test-context

A test context is required to tell on what you want to run the tests. Currenly Insights only support "ProfileTestContext"


In [4]:
from mlm_insights.tests.test_context.profile_test_context import ProfileTestContext
profile_test_context = ProfileTestContext(current_profile = current_profile, reference_profile=reference_profile)

## Creating tests using JSON file
Test configuration can be part of existing monitor config json file or a seperate json file. The json file must contains 'test_config' section. In the below example we are using the same json file which was using to create the current profile.

In [5]:
test_builder = InsightsConfigReader(config_location=monitor_config_path).get_test_builder(profile_test_context).build()
test_results_using_config = test_builder.run()

In [6]:
print(test_results_using_config.test_summary)

TestSummary(total_tests=5, passed_tests=3, failed_tests=2, error_test=0)


In [7]:
def formatted_result(test_results):
    formatted_result = list(map(lambda x: [x.description, x.status.name], test_results))
    print(*formatted_result, sep='\n\n')
    
formatted_result(test_results_using_config.test_results)

['The Min of feature Pregnancies is 1.0. Test condition : 1.0 > 4.5', 'FAILED']

['The Mean of feature Pregnancies is 3.8422174840085286. The Mean of feature Pregnancies is 3.8422174840085286 in reference profile. Test condition : 3.8422174840085286 deviates by +/- 10.0% from 3.8422174840085286', 'PASSED']

['The Variance of feature Age is 144.83936697869166. Test condition : 144.83936697869166 < 1.5', 'FAILED']

['TestIsComplete: Completion percentage of feature Age is 100.0. Completion percentage of feature Age is 100.0 in reference profile. Test condition: 100.0 >= 100.0', 'PASSED']

["TestNoNewCategory: Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight']. Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'] in reference profile. Test condition: ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'] = ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight']", 'PASSED']


## Writing tests using Test Builder

Insights also provides builder interface to write test. This allow user to write more grananular test, custom tests.

In [8]:
from mlm_insights.tests.test_builder.test_builder import InsightsTestBuilder

from mlm_insights.tests.test_types.metric_based_tests.test_is_complete import TestIsComplete
from mlm_insights.tests.test_types.metric_based_tests.test_is_positive import TestIsPositive
from mlm_insights.tests.test_types.predicate_based_tests.interfaces.predicate_based_test_base import \
    PredicateBasedTestBase
from mlm_insights.tests.test_types.predicate_based_tests.test_equal import TestEqual
from mlm_insights.tests.test_types.predicate_based_tests.test_greater_than import TestGreaterThan
from mlm_insights.tests.test_types.predicate_based_tests.test_less_than import TestLessThan

from mlm_insights.tests.selectors.feature_metric_selector import FeatureMetricSelector
from mlm_insights.tests.selectors.dataset_metric_selector import DatasetMetricSelector

from mlm_insights.tests.profile_source import ProfileSource

test_builder = InsightsTestBuilder().with_context(profile_test_context) \
        .add_test(TestGreaterThan(
        lhs=FeatureMetricSelector(profile_source=ProfileSource.CURRENT,
                                  feature_name="Pregnancies",
                                  metric_key="Mean"),
        rhs=10.0)) \
        .add_test(TestLessThan(
        lhs=FeatureMetricSelector(profile_source=ProfileSource.CURRENT,
                                  feature_name="Pregnancies",
                                  metric_key="Mean"),
        rhs=FeatureMetricSelector(profile_source=ProfileSource.CURRENT,
                                  feature_name="Pregnancies",
                                  metric_key="Quartiles.q2"))) \
        .add_test(TestEqual(
        lhs=DatasetMetricSelector(profile_source=ProfileSource.CURRENT,
                                  metric_key="RowCount"),
        rhs=1000)) \
        .add_test(TestIsPositive(feature_name="Age")) \
        .add_test(TestIsComplete(feature_name="BMICategory", threshold_value=90)).build()

In [9]:
test_results_using_builder = test_builder.run()
formatted_result(test_results_using_builder.test_results)

['The Mean of feature Pregnancies is 3.8422174840085286. Test condition : 3.8422174840085286 > 10.0', 'FAILED']

['The Mean of feature Pregnancies is 3.8422174840085286. The Quartiles.q2 of feature Pregnancies is 3.0. Test condition : 3.8422174840085286 < 3.0', 'FAILED']

['The RowCount is 469.0. Test condition : 469.0 = 1000', 'FAILED']

['TestIsPositive: Feature Age has all positive values', 'PASSED']

['TestIsComplete: Completion percentage of feature BMICategory is 100.0. Threshold value is set to 90. Test condition: 100.0 >= 90', 'PASSED']


## Test Result Grouping

It is possible to group the test result by :
1. Feature
2. Test status
3. Custom tags.

In [10]:
from mlm_insights.tests.constants import GroupByKey
grouped_result = test_results_using_config.group_tests_by(GroupByKey.FEATURE_TAG_KEY)

for key, value in grouped_result.items():
    print(f"Test for Feature = {key}")
    formatted_result(value)
    print()

Test for Feature = Pregnancies
['The Min of feature Pregnancies is 1.0. Test condition : 1.0 > 4.5', 'FAILED']

['The Mean of feature Pregnancies is 3.8422174840085286. The Mean of feature Pregnancies is 3.8422174840085286 in reference profile. Test condition : 3.8422174840085286 deviates by +/- 10.0% from 3.8422174840085286', 'PASSED']

Test for Feature = Age
['The Variance of feature Age is 144.83936697869166. Test condition : 144.83936697869166 < 1.5', 'FAILED']

['TestIsComplete: Completion percentage of feature Age is 100.0. Completion percentage of feature Age is 100.0 in reference profile. Test condition: 100.0 >= 100.0', 'PASSED']

Test for Feature = BMICategory
["TestNoNewCategory: Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight']. Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'] in reference profile. Test condition: ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'

In [11]:
from mlm_insights.tests.constants import GroupByKey
grouped_result = test_results_using_config.group_tests_by(GroupByKey.TEST_STATUS_KEY)

for key, value in grouped_result.items():
    print(f"Test Status = {key}")
    formatted_result(value)
    print()

Test Status = FAILED
['The Min of feature Pregnancies is 1.0. Test condition : 1.0 > 4.5', 'FAILED']

['The Variance of feature Age is 144.83936697869166. Test condition : 144.83936697869166 < 1.5', 'FAILED']

Test Status = PASSED
['The Mean of feature Pregnancies is 3.8422174840085286. The Mean of feature Pregnancies is 3.8422174840085286 in reference profile. Test condition : 3.8422174840085286 deviates by +/- 10.0% from 3.8422174840085286', 'PASSED']

['TestIsComplete: Completion percentage of feature Age is 100.0. Completion percentage of feature Age is 100.0 in reference profile. Test condition: 100.0 >= 100.0', 'PASSED']

["TestNoNewCategory: Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight']. Category values of feature BMICategory are ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'] in reference profile. Test condition: ['Obesity', 'Overweight', 'Healthy Weight', 'Underweight'] = ['Obesity', 'Overweight', 'Healthy Weig